In [3]:
from langchain_chroma import Chroma

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
import pandas as pd
hadiths = pd.read_csv("csv/hadith.csv")

In [8]:
hadiths

,id,arabic,translate_id,narator,hadith_scholar,source,hadith_no,level_of_hadith
0,0,"فِي رِوَايَتِهِ ""إِذَا وَلَدَتِ الأَمَةُ بَعْل...","“Dalam riwayat lain, ""... ketika para al-amatu...",Abu Hurairah,Muslim bin Hajjaj,Shahih Muslim,6,Shahih
1,1,عَنْ يَحْيَى بْنِ يَعْمَرَ، عَنِ ابْنِ عُمَرَ،...,"“Dari Yahya bin Ya'mar dari Ibnu Umar, dari Um...",Abdullah bin Umar,Muslim bin Hajjaj,Shahih Muslim,4,Shahih
2,2,عن يحيى بن يَعْمَرَ، وَحُمَيْدِ بْنِ عَبْدِ ال...,“Dari Yahya bin Ya'mar dan Humaid bin 'Abdur-R...,Abdullah bin Umar,Muslim bin Hajjaj,Shahih Muslim,3,Shahih
3,3,عَنْ يَحْيَى بْنِ يَعْمَرَ قَالَ لَمَّا تَكَلّ...,"“Yahya bin Ya'mar berkata: ""Ketika Ma'bad mula...",Abdullah bin Umar,Muslim bin Hajjaj,Shahih Muslim,2,Shahih
4,4,كَانَ أَوَّلَ مَنْ قَالَ فِي الْقَدَرِ بِالْبَ...,“Orang yang pertama kali membahas takdir di Ba...,Abdullah bin Umar,Muslim bin Hajjaj,Shahih Muslim,1,Shahih
...,...,...,...,...,...,...,...,...
778,778,مَن سَمِعَ النِّداءَ فلَم يأتِ فلا صَلاةَ لَه ...,"“Barangsiapa yang mendengar adzan, namun tidak...",Abdullah bin Abbas,Ibnu Hajar Al Asqalani,Bulughul Maram,114,Shahih
779,779,إنَّ لك من الأجرِ على قدْرِ نَصَبِك و نفقتِك,“Sesungguhnya besar pahala sebanding dengan ke...,Aisyah,Muhammad Nashiruddin Al Albani,Shahih Jami' Ash Shaghir,2160,Shahih
780,780,يَوَدُّ أَهلُ العَافِيَةِ يَومَ القِيَامَةِ حِ...,“Orang-orang yang dahulu sehat di dunia kelak ...,Jabir bin Abdillah,Muhammad Nashiruddin Al Albani,Shahih At Tirmidzi,2402,Shahih
781,781,: خطبنا رسول الله صلى الله عليه وسلم، فأقبل ال...,“Rasulullah Shallallahu ‘alaihi wa sallam berk...,Abdullah bin Buraidah,Muhammad Nashiruddin Al Albani,Shahih Abu Daud,1109,Shahih


In [9]:
ids = hadiths["id"].astype(str).tolist()
texts = hadiths["translate_id"].tolist()
metadatas = hadiths.apply(
    lambda x: {
        "id": x["id"],
        "arabic": x["arabic"],
        "translate_id": x["translate_id"],
        "narrator": x["narator"],
        "source": x["source"],
        "hadith_no": x["hadith_no"],
    },
    axis=1,
).tolist()

In [10]:
from sentence_transformers import SentenceTransformer

In [11]:
from langchain.embeddings.base import Embeddings
from typing import List
class SentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name: str):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, documents: List[str]) -> List[List[float]]:
        return self.model.encode(documents)

    def embed_query(self, query: str) -> List[float]:
        return self.model.encode([query])[0]

In [12]:
db = Chroma.from_texts(texts=texts, metadatas=metadatas, embedding=SentenceTransformerEmbeddings("indolem/indobert-base-uncased"))

No sentence-transformers model found with name indolem/indobert-base-uncased. Creating a new one with mean pooling.


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/234k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [16]:
query = "Tata cara salat"
docs = db.similarity_search(query, k=10)
docs

[Document(id='585d8848-1ade-48fc-b7bb-b39647223666', metadata={'arabic': 'أكْثرُ مُنافقي أمَّتي قرَّاؤُها', 'hadith_no': '10/125', 'id': 224, 'narrator': "Abdullah bin 'Amr bin Al Ash", 'source': 'Takhrij Musnad Ahmad', 'translate_id': "“Kebanyakan orang munafik dari kalangan umatku adalah para pembaca Al Qur'an”"}, page_content="“Kebanyakan orang munafik dari kalangan umatku adalah para pembaca Al Qur'an”"),
 Document(id='8e154eaf-5b6c-4bba-aa28-2edd12d89adf', metadata={'arabic': 'أكْثرُ مُنافقي أمَّتي قرَّاؤُها', 'hadith_no': '750', 'id': 223, 'narrator': "Abdullah bin 'Amr bin Al Ash", 'source': 'As Silsilah Al Ahadits Ash Shahihah', 'translate_id': "“Kebanyakan orang munafik dari kalangan umatku adalah para pembaca Al Qur'an”"}, page_content="“Kebanyakan orang munafik dari kalangan umatku adalah para pembaca Al Qur'an”"),
 Document(id='770a5e6b-1338-48fc-bbaf-a25effdac9ee', metadata={'arabic': 'مِنْ خُلَفائِكُمْ خَلِيفَةٌ يَحْثُو المالَ حَثْيًا، لا يَعُدُّهُ عَدَدًا. وفي رِوايَةِ ا